# Test of DQN

In [1]:
%run DQN

DQN


Test that the imports went smoothly.

In [2]:
bar = nn.MaxPool2d(kernel_size = 1)
bar(torch.randn(1, 2, 3)) # batch of 1, 2 rows, 3 cols
bar(torch.randn(1, 3, 2)) # batch of 1, 3 rows, 2 cols

tensor([[[-1.4567, -0.1631],
         [ 0.6478,  0.7036],
         [ 0.2480, -0.3151]]])

Calculating output of convolution sizes.

In [3]:
def convSize(W_or_H, K, P, S):
    return( (W_or_H - K + (2 * P)) / S + 1)

In [4]:
for i in range(0, 3):
    print(convSize( 256 / (i + 1), 4, 1, 2))

128.0
64.0
42.666666666666664


In [5]:
for i in range(0, 3):
    print(convSize( 240 / (i + 1), 4, 1, 2))

120.0
60.0
40.0


The padding is problematic for the conventional height.

The use of "Adaptive" pooling seems like a convenient way to keep it simple.

Demonstration below.

In [6]:
#nn.AdaptiveAvgPool2d?

In [18]:
torch.manual_seed(5004 * 2040);

In [8]:
rando = torch.randn(1, 1, 4)
m = nn.MaxPool1d(kernel_size = 4)
am = nn.AdaptiveMaxPool1d(output_size = 1)
av = nn.AdaptiveAvgPool2d(output_size = 1)

print(rando)
print(m(rando))
print(am(rando))
print(av(rando))

tensor([[[0.5552, 0.2258, 1.4282, 0.3485]]])
tensor([[[1.4282]]])
tensor([[[1.4282]]])
tensor([[[0.6394]]])


In [9]:
rando = torch.randn(1, 1, 2, 2)
m = nn.MaxPool2d(kernel_size = (2, 2))
am = nn.AdaptiveMaxPool2d(output_size = (1, 1))
av = nn.AdaptiveAvgPool2d(output_size = (1, 1))

print(rando)
print(m(rando))
print(am(rando))
print(av(rando))

tensor([[[[-0.0779, -0.9177],
          [-0.7388, -0.8794]]]])
tensor([[[[-0.0779]]]])
tensor([[[[-0.0779]]]])
tensor([[[[-0.6535]]]])


In [10]:
rando = torch.randn(1, 1, 3, 3)
m = nn.MaxPool2d(kernel_size = (2, 2))
am = nn.AdaptiveMaxPool2d(output_size = (2, 2))
av = nn.AdaptiveAvgPool2d(output_size = (2, 2))

print(rando)
print(m(rando))
print(am(rando))
print(av(rando))

tensor([[[[ 0.9801,  0.4852,  0.2811],
          [-1.1147, -0.4985, -1.3156],
          [-1.0394,  1.0127,  1.0207]]]])
tensor([[[[0.9801]]]])
tensor([[[[0.9801, 0.4852],
          [1.0127, 1.0207]]]])
tensor([[[[-0.0370, -0.2619],
          [-0.4100,  0.0548]]]])


Test of foo methods.

Initialization.

In [19]:
foo = DQN(actionSpaceSize = 4, RGB = True)

In [20]:
foo.network

Sequential(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2))
  (1): AdaptiveMaxPool2d(output_size=(128, 120))
  (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
  (3): AdaptiveMaxPool2d(output_size=(64, 60))
  (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2))
  (5): AdaptiveAvgPool2d(output_size=(32, 30))
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=61440, out_features=16, bias=True)
  (8): ReLU()
  (9): Linear(in_features=16, out_features=4, bias=True)
)

Without prompting, the weights are being randomly initialized.

In [21]:
foo.network[0].weight[0]

tensor([[[ 0.1726, -0.1243,  0.0812],
         [ 0.1095, -0.1161,  0.0540],
         [ 0.0835,  0.1737, -0.1866]],

        [[-0.1881,  0.0004,  0.1914],
         [-0.0726, -0.0337,  0.1578],
         [ 0.0084,  0.1223,  0.0363]],

        [[-0.0018, -0.0553,  0.0405],
         [-0.1047, -0.1314,  0.0109],
         [-0.1259, -0.0868,  0.0598]]], grad_fn=<SelectBackward0>)

Forward pass

In [22]:
foo.forward(torch.randn(1, 3, 256, 240))

tensor([[-0.0543,  0.1541,  0.1534, -0.1386]], grad_fn=<AddmmBackward0>)

Save and load

In [23]:
foo.saveModel("./savedModels/foo.pth")

In [24]:
foo.loadModel("./savedModels/foo_old.pth")

In [25]:
foo.network[0].weight[0]

tensor([[[-0.0043, -0.1761, -0.0093],
         [ 0.0445,  0.0724,  0.1455],
         [ 0.0138,  0.0723, -0.1406]],

        [[ 0.1770, -0.0852, -0.1290],
         [-0.1239, -0.1760, -0.0494],
         [-0.1182,  0.1922,  0.0362]],

        [[-0.0359, -0.0513, -0.1751],
         [ 0.1220, -0.1007,  0.0274],
         [ 0.1366,  0.1569,  0.0464]]], grad_fn=<SelectBackward0>)